## A minimalist example for recovering sparse graphs using `mGLAD`

Fitting meta-GLAD on a batch of erdos-renyi random sparse graphs with samples obtained from a corresponding multivariate Gaussian distribution  

### About `mGLAD` 
A meta learning based approach to recover sparse graphs. This work proposes `mGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key Benefits & features:  
- It is a fast alternative to solving the Graphical Lasso problem as
    - GPU based acceleration can be leveraged
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization).     
- mGLAD automatically learns the sparsity related regularization parameters. This gives an added benefit to mGLAD over other graphical lasso solvers.  
- The meta loss is the logdet objective of the graphical lasso `1/B(-1*log|theta|+ <S, theta>)`, where `B=batch_size, S=input covariance matrix, theta=predicted precision matrix`.   

In [ ]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
torch.__version__

'1.9.0'

### Create Synthetic data

In [3]:
from scripts import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, trueTheta = main.getGLADdata(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=2
)    
print(f'TrueTheta: {trueTheta.shape}, Samples {Xb.shape}')

TrueTheta: torch.Size([2, 10, 10]), Samples torch.Size([2, 500, 10])


### Running the GLAD-Meta algorithm
Until Convergence:  

    1. Initialize learnable `GLAD` parameters
    2. Run the GLAD model
    3. Get the meta-loss
    4. Backprop

Possible reasons if `mGLAD` does not converge: 

    1. Please re-run. This will run the optimization with different initializations  
    2. Lower the learning rate  
    3. Change the INIT_DIAG=0/1 in the `GLAD` model parameters

In [4]:
# main source file for meta-GLAD

# Helper functions for DAG Meta
from scripts.glad.glad_model import glad_params
from scripts.glad import glad
from scripts.main import lossGLADmeta
from scripts.utils import prepare_data
from scripts.utils.metrics import reportMetrics

from pprint import pprint

def initGLAD():
    """
    Initialize the GLAD model parameters and the optimizer
    to be used.
    """
    model = glad_params(theta_init_offset=0.1, nF=3, H=3)
    optimizer = glad.get_optimizers(model)
    return model, optimizer

def runGLAD(Sb, model_glad):
    """Run the input through the GLAD meta algorithm.
    It executes the following steps in batch mode
    1. Run the GLAD model to get initial good regularization
    2. Calculate the meta-loss
    
    Args:
        Sb (torch.Tensor BxDxD): The input covariance matrix
        metaGLADmodel (dict): Contains the learnable params
    
    Returns:
        loss (torch.scalar): The meta loss 
        predTheta (torch.Tensor BxDxD): The predicted theta
    """
    # 1. Running the GLAD model 
    predTheta = glad.glad(Sb, model_glad)
    # 2. Calculate the meta-loss
    loss = lossGLADmeta(predTheta, Sb)
    return loss, predTheta


def GLAD_Meta_main(Xb, trueTheta=None, EPOCHS=50):
    """Running the DAG Meta algorithm.
    
    Args:
        Xb (torch.Tensor BxMxD): The input sample matrix
        trueDAG (torch.Tensor BxDxD): The corresponding 
            true DAGs for reporting metrics.
        EPOCHS (int): The number of training epochs
        
    """
    # Calculating the batch covariance
    B, M, D = Xb.shape
    Sb = prepare_data.getCovariance(Xb) # BxDxD
    # optimizer and model for GLAD
    model_glad, optimizer_glad = initGLAD()    
    # Optimizing for the meta loss
    for e in range(EPOCHS):      
        # reset the grads to zero
        optimizer_glad.zero_grad()
        # calculate the loss
        loss, predTheta = runGLAD(Sb, model_glad)
        # calculate the backward gradients
        loss.backward()
        print(f'epoch:{e}/{EPOCHS} loss:{loss.detach().numpy()}')
        # updating the optimizer params with the grads
        optimizer_glad.step()
        # reporting the metrics if true DAGs provided
        if trueTheta is not None and (e+1)%EPOCHS == 0:
            for b in range(B):
                compare_theta = reportMetrics(
                    trueTheta[b].detach().numpy(), 
                    predTheta[b].detach().numpy()
                )
                print(f'Batch:{b} - {compare_theta}')

In [5]:
GLAD_Meta_main(Xb, trueTheta)

epoch:0/50 loss:15.626142501831055
epoch:1/50 loss:14.745260238647461
epoch:2/50 loss:14.628384590148926
epoch:3/50 loss:14.539222717285156
epoch:4/50 loss:14.08952522277832
epoch:5/50 loss:13.352099418640137
epoch:6/50 loss:12.430947303771973
epoch:7/50 loss:11.43539810180664
epoch:8/50 loss:10.578899383544922
epoch:9/50 loss:9.953746795654297
epoch:10/50 loss:9.516615867614746
epoch:11/50 loss:9.219841957092285
epoch:12/50 loss:9.005231857299805
epoch:13/50 loss:8.861539840698242
epoch:14/50 loss:8.76992130279541
epoch:15/50 loss:8.712407112121582
epoch:16/50 loss:8.680265426635742
epoch:17/50 loss:8.668645858764648
epoch:18/50 loss:8.674535751342773
epoch:19/50 loss:8.676127433776855
epoch:20/50 loss:8.659995079040527
epoch:21/50 loss:8.632818222045898
epoch:22/50 loss:8.613653182983398
epoch:23/50 loss:8.604607582092285
epoch:24/50 loss:8.599124908447266
epoch:25/50 loss:8.595662117004395
epoch:26/50 loss:8.592811584472656
epoch:27/50 loss:8.590316772460938
epoch:28/50 loss:8.58831